INIT enthält die initialisierung die von jedem auswerter geladen wird:      
 + setzen der Intervalle   
 + laden der files btc, eur, s&p, nasdaq, gold, oel, treasury_10Y 
 + rechnen der returns
 + erstellen von concatenates 

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import edhec_risk_kit_129 as erk
import numpy as np
import pandas as pd
import scipy.stats
import seaborn as sns
import statsmodels.api as sm
from matplotlib import pyplot as plt 
from scipy import stats

Goldpreis WKN: 965515 ISIN: XC0009655157
https://www.ariva.de/goldpreis-gold-kurs/historische_kurse


Brent Crude Ölpreis Historische Kurse   ISIN: XC0009677409
https://www.ariva.de/brent_crude_roh%C3%B6l_ice_rolling-kurs/historische_kurse
    
https://de.finance.yahoo.com/quote/EURUSD%3DX/history  
BTC-USD  
EUR-USD
^GSPC  
^IXIC  
10-Year T-Note Futures,Sep-2020 (ZN=F)
( yield not used : Treasury Yield 10 Years (^TNX) )


In [ ]:
dat_start  = "2017-01-03"   #"2014-09-11"   #2018-02
dat_end    = "2020-07-10"   #2020-02

iv1_start = "2018-02-01"
iv1_end   = "2020-02-01"

iv2_start = "2018-02-01"
iv2_end   = "2020-07-10"

btc_file = "BTC-USD_day"
eur_file = "EUR-USD_day"
s_p_file = "^GSPC_S&P_day"
ndq_file = "^IXIC_nasdaq_day"
gld_file = "wkn_965515_gold"  #"GC=F_gold_day"
oil_file = "wkn_COM062_oel"   #CL-F_crudeoil
trs_file = "ZN=F_10Y_day"       #"^TNX_10Y_day"

BTC = "bitcoin"
EUR = "eur"
S_P = "S_P"
NDQ = "Nasdaq"
GLD = "Gold"
OIL = "Oil"
TRS = "Treasury"

roll_size      = 250 #250 ist gaengig / 257=1y  #66=3m
roll_size_week = 52

### Initialisierung
Laden daily bitcoin, S&P .... 
Ermitteln der Returns. 
Die folgenden rolling Auswertungen beziehen sich auf den Beobachtungszeitraum.

In [ ]:
btc0 = pd.read_csv("_data/" + btc_file + ".csv", header=0, index_col=0)
eur0 = pd.read_csv("_data/" + eur_file + ".csv", header=0, index_col=0)
s_p0 = pd.read_csv("_data/" + s_p_file + ".csv", header=0, index_col=0)
ndq0 = pd.read_csv("_data/" + ndq_file + ".csv", header=0, index_col=0)
gld0 = pd.read_csv("_data/" + gld_file + ".csv", header=0, index_col=0)
oil0 = pd.read_csv("_data/" + oil_file + ".csv", header=0, index_col=0)
trs0 = pd.read_csv("_data/" + trs_file + ".csv", header=0, index_col=0).dropna()

btc_cls = btc0[dat_start : dat_end]['Close'].to_frame()
btc_cls.rename(columns={'Close':BTC}, inplace=True)

eur_cls = eur0[dat_start : dat_end]['Close'].to_frame()
eur_cls.rename(columns={'Close':EUR}, inplace=True)

s_p_cls = s_p0[dat_start : dat_end]['Close'].to_frame()
s_p_cls.rename(columns={'Close':S_P}, inplace=True)

ndq_cls = ndq0[dat_start : dat_end]['Close'].to_frame()
ndq_cls.rename(columns={'Close':NDQ}, inplace=True)

gld_cls = gld0[dat_start : dat_end]['Schlusskurs'].to_frame()
gld_cls.rename(columns={'Schlusskurs':GLD}, inplace=True)

oil_cls = oil0[dat_start : dat_end]['Schlusskurs'].to_frame()
oil_cls.rename(columns={'Schlusskurs':OIL}, inplace=True)

trs_cls = trs0[dat_start : dat_end]['Close'].to_frame()
trs_cls.rename(columns={'Close':TRS}, inplace=True)

# btc_cls.index = pd.to_datetime(btc_cls.index).to_period('W-MON')
# oth_cls.index = pd.to_datetime(oth_cls.index).to_period('W-MON')
all_cls = pd.concat([btc_cls, eur_cls, s_p_cls, ndq_cls, gld_cls, oil_cls, trs_cls], axis=1, sort=True).dropna()

#btc_r05 = btc_ret * 0.5
#btc_r05.rename(columns={btc_nam : btc_nam+'*0.5'}, inplace=True)
btc_ret = btc_cls.pct_change().dropna()
eur_ret = eur_cls.pct_change().dropna()
s_p_ret = s_p_cls.pct_change().dropna()
ndq_ret = ndq_cls.pct_change().dropna()
gld_ret = gld_cls.pct_change().dropna()
oil_ret = oil_cls.pct_change().dropna()
trs_ret = trs_cls.pct_change().dropna()

all_ret = pd.concat([btc_ret, eur_ret, s_p_ret, ndq_ret, gld_ret, oil_ret, trs_ret], axis=1, sort=True).dropna()
#all_ret = pd.concat([btc_ret, btc_r05, s_p_ret, gld_ret, oil_ret], axis=1, sort=True).dropna()

iv1_cls  = all_cls[iv1_start : iv1_end]
iv1_ret  = all_ret[iv1_start : iv1_end]

iv2_cls  = all_cls[iv2_start : iv2_end]
iv2_ret  = all_ret[iv2_start : iv2_end]

In [ ]:
print ("btc_cls : " , btc_cls.shape)
print ("eur_cls : " , eur_cls.shape)
print ("s_p_cls : " , s_p_cls.shape)
print ("ndq_cls : " , ndq_cls.shape)
print ("gld_cls : " , gld_cls.shape)
print ("oil_cls : " , oil_cls.shape)
print ("trs_cls : " , oil_cls.shape)
print ("all_cls : " , all_cls.shape)

In [ ]:
print ("btc_ret : " , btc_ret.shape)
print ("eur_ret : " , eur_ret.shape)
print ("s_p_ret : " , s_p_ret.shape)
print ("ndq_ret : " , ndq_ret.shape)
print ("gld_ret : " , gld_ret.shape)
print ("oil_ret : " , oil_ret.shape)
print ("trs_ret : " , oil_ret.shape)
print ("all_ret : " , all_ret.shape)

In [ ]:
#all_cls[btc_nam].plot(figsize=(9, 6)) 
#all_cls[oil_nam].plot(figsize=(9, 6)) 
#iv2_cls[gld_nam].plot(figsize=(9, 6)) 

#iv1_cls[[btc_nam, s_p_nam]].plot(figsize=(9, 6)) 
#iv2_ret[[btc_nam, s_p_nam]].plot(figsize=(9, 6)) 

In [ ]:
#all_cls[s_p_nam].plot(figsize=(9, 6)) 

In [ ]:
#pd.set_option('display.max_rows', None)
#pd.concat([btc_cls, eur_cls, s_p_cls, oil_cls, trs_cls, all_cls], axis=1, sort=True)
#pd.concat([btc_ret, eur_ret, s_p_ret, oil_ret, trs_ret, all_ret], axis=1, sort=True)

In [ ]:
all_cls_fri1 = all_cls # [iv1_start : ]
all_cls_fri1.index = pd.to_datetime(all_cls_fri1.index)
all_cls_fri = all_cls_fri1[(all_cls_fri1.index.weekday==4) | (all_cls_fri1.index == '2018-03-08') 
                                                           | (all_cls_fri1.index == '2018-03-29') 
                                                           | (all_cls_fri1.index == '2018-11-21')
                                                           | (all_cls_fri1.index == '2019-04-18')
                                                           | (all_cls_fri1.index == '2019-11-27')
                                                           | (all_cls_fri1.index == '2020-04-09')
                                                           | (all_cls_fri1.index == '2020-04-30')
                                                           | (all_cls_fri1.index == '2020-07-02')]  #3
all_ret_fri = all_cls_fri.pct_change().dropna()
#print ("all_cls_fri : " , all_cls_fri.shape)
print ("all_ret_fri : " , all_ret_fri.shape)

In [ ]:
#pd.set_option('display.max_rows', None)
#all_ret
#all_ret.describe()

In [ ]:
#all_ret_fri.describe()